### 使用对话链调用memory

In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.conversation.base import ConversationChain
from langchain .prompts import PromptTemplate


#添加记忆模块
from langchain.memory import ConversationBufferMemory

import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("api_key")
base_url = os.getenv("base_url")

In [2]:
online_llm = ChatOpenAI(
    api_key=api_key,
    model="qwen-plus",
    base_url=base_url,
    temperature=0
)

In [3]:
##初始化记忆模块
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

/var/folders/xh/fl5j9g1n5t79htj8yk1cswzh0000gn/T/ipykernel_15675/2315687189.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [4]:
#需要将记忆模块中的 关键 memory_key 添加到提示词中，大模型对话时无法使用记忆内容
template = '''
你是一个全能助手，可以根据用户的问题给出合适的答案。对话历史如下：{chat_history}。问题如下：{question}
'''

prompt = PromptTemplate(
    input_variables=['chat_history',"question"],
    template=template
)

In [ ]:
# ConversationChain.prompt = prompt

In [ ]:
# ConversationChain.prompt

PromptTemplate(input_variables=['chat_history', 'question'], input_types={}, partial_variables={}, template='\n你是一个全能助手，可以根据用户的问题给出合适的答案。对话历史如下：{chat_history}。问题如下：{question}\n')

In [5]:
#定义对话链
llm_chain = ConversationChain(
    llm=online_llm,
    prompt=prompt,
    memory=memory,
    verbose=True,
    input_key='question'
)

/var/folders/xh/fl5j9g1n5t79htj8yk1cswzh0000gn/T/ipykernel_15675/4143181577.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  llm_chain = ConversationChain(


In [6]:
llm_chain.prompt.template

'\n你是一个全能助手，可以根据用户的问题给出合适的答案。对话历史如下：{chat_history}。问题如下：{question}\n'

In [16]:
result = llm_chain.invoke("你是谁？")
print(result)



> Entering new ConversationChain chain...
Prompt after formatting:

你是一个全能助手，可以根据用户的问题给出合适的答案。对话历史如下：[HumanMessage(content='你是谁？', additional_kwargs={}, response_metadata={}), AIMessage(content='我是通义千问，阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够回答问题、创作文字，比如写故事、公文、邮件、剧本等，还能进行逻辑推理、编程，甚至表达观点和玩游戏。我在多国语言上都有很好的掌握，能为你提供多样化的帮助。有什么我可以帮到你的吗？', additional_kwargs={}, response_metadata={})]。问题如下：你是谁？


> Finished chain.
{'question': '你是谁？', 'chat_history': [HumanMessage(content='你是谁？', additional_kwargs={}, response_metadata={}), AIMessage(content='我是通义千问，阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够回答问题、创作文字，比如写故事、公文、邮件、剧本等，还能进行逻辑推理、编程，甚至表达观点和玩游戏。我在多国语言上都有很好的掌握，能为你提供多样化的帮助。有什么我可以帮到你的吗？', additional_kwargs={}, response_metadata={}), HumanMessage(content='你是谁？', additional_kwargs={}, response_metadata={}), AIMessage(content='我是通义千问，阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够回答问题、创作文字，比如写故事、公文、邮件、剧本等，还能进行逻辑推理、编程，甚至表达观点和玩游戏。我在多国语言上都有很好的掌握，能为你提供多样化的帮助。有什么我可以帮到你的吗？', additional_kwargs={}, response_metadata={})], 'response': '我是通义千问，阿里巴巴集团

In [17]:
#查看对话历史 方式一
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='你是谁？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='我是通义千问，阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够回答问题、创作文字，比如写故事、公文、邮件、剧本等，还能进行逻辑推理、编程，甚至表达观点和玩游戏。我在多国语言上都有很好的掌握，能为你提供多样化的帮助。有什么我可以帮到你的吗？', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='你是谁？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='我是通义千问，阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够回答问题、创作文字，比如写故事、公文、邮件、剧本等，还能进行逻辑推理、编程，甚至表达观点和玩游戏。我在多国语言上都有很好的掌握，能为你提供多样化的帮助。有什么我可以帮到你的吗？', additional_kwargs={}, response_metadata={})]}

In [9]:
#查看对话历史 方式二
llm_chain.memory.chat_memory.messages

[HumanMessage(content='你是谁？', additional_kwargs={}, response_metadata={}),
 AIMessage(content='我是通义千问，阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够回答问题、创作文字，比如写故事、公文、邮件、剧本等，还能进行逻辑推理、编程，甚至表达观点和玩游戏。我在多国语言上都有很好的掌握，能为你提供多样化的帮助。有什么我可以帮到你的吗？', additional_kwargs={}, response_metadata={})]

In [10]:
#查看对话历史 方式三
memory.chat_memory.messages

[HumanMessage(content='你是谁？', additional_kwargs={}, response_metadata={}),
 AIMessage(content='我是通义千问，阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够回答问题、创作文字，比如写故事、公文、邮件、剧本等，还能进行逻辑推理、编程，甚至表达观点和玩游戏。我在多国语言上都有很好的掌握，能为你提供多样化的帮助。有什么我可以帮到你的吗？', additional_kwargs={}, response_metadata={})]

### 记忆存储到本地

In [11]:
from langchain.schema import messages_to_dict, messages_from_dict

dict_memory = messages_to_dict(llm_chain.memory.chat_memory.messages)
print("---------------dict_memory---------------")
print(dict_memory)

---------------dict_memory---------------
[{'type': 'human', 'data': {'content': '你是谁？', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': None, 'example': False}}, {'type': 'ai', 'data': {'content': '我是通义千问，阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够回答问题、创作文字，比如写故事、公文、邮件、剧本等，还能进行逻辑推理、编程，甚至表达观点和玩游戏。我在多国语言上都有很好的掌握，能为你提供多样化的帮助。有什么我可以帮到你的吗？', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'ai', 'name': None, 'id': None, 'example': False, 'tool_calls': [], 'invalid_tool_calls': [], 'usage_metadata': None}}]


In [12]:
#上下文记忆存入本地文件
import pickle
with open("memory.pkl", "wb") as f:
    pickle.dump(dict_memory, f)
print("上下文记忆已存入本地文件")

上下文记忆已存入本地文件


### 清除 对话链中 上下文记忆

In [13]:
#清除上下文记忆
llm_chain.memory.clear()
print("上下文记忆已清除")
print(llm_chain.memory.chat_memory.messages)

上下文记忆已清除
[]


### 从本地加载记忆

In [14]:
#上下文记忆从本地文件读取
with open("memory.pkl", "rb") as f:
    dict_memory2 = pickle.load(f)
print(dict_memory2)
print("上下文记忆已从本地文件读取")

messages = messages_from_dict(dict_memory2)
messages

[{'type': 'human', 'data': {'content': '你是谁？', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': None, 'example': False}}, {'type': 'ai', 'data': {'content': '我是通义千问，阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够回答问题、创作文字，比如写故事、公文、邮件、剧本等，还能进行逻辑推理、编程，甚至表达观点和玩游戏。我在多国语言上都有很好的掌握，能为你提供多样化的帮助。有什么我可以帮到你的吗？', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'ai', 'name': None, 'id': None, 'example': False, 'tool_calls': [], 'invalid_tool_calls': [], 'usage_metadata': None}}]
上下文记忆已从本地文件读取


[HumanMessage(content='你是谁？', additional_kwargs={}, response_metadata={}),
 AIMessage(content='我是通义千问，阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够回答问题、创作文字，比如写故事、公文、邮件、剧本等，还能进行逻辑推理、编程，甚至表达观点和玩游戏。我在多国语言上都有很好的掌握，能为你提供多样化的帮助。有什么我可以帮到你的吗？', additional_kwargs={}, response_metadata={})]

### 将本地记忆加载到对话链中

In [15]:
#上下文记忆的加载方式
llm_chain.memory.chat_memory.messages = messages
print(llm_chain.memory.chat_memory.messages)

[HumanMessage(content='你是谁？', additional_kwargs={}, response_metadata={}), AIMessage(content='我是通义千问，阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够回答问题、创作文字，比如写故事、公文、邮件、剧本等，还能进行逻辑推理、编程，甚至表达观点和玩游戏。我在多国语言上都有很好的掌握，能为你提供多样化的帮助。有什么我可以帮到你的吗？', additional_kwargs={}, response_metadata={})]
